In [ ]:
import pickle
from os import path

#@formatter:off
from AnalysisFunctions import analyze_statistics, print_exception_tracebacks_for_analyzer, \
    print_exception_tracebacks, plot_sim_times, plot_exec_times, plot_num_obstacles, plot_scenarios
#@formatter:on
from BatchConversion.Serializable import Serializable
from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer

#@formatter:off
%matplotlib inline
#@formatter:on

In [ ]:
stats_dir = "results/"
do_plot_scenarios = False

In [ ]:
with open(path.join(stats_dir, "statistics.pickle"), "rb") as stats_file:
    # We don't need the scenario files for this analysis, just slows us down
    Serializable.import_extra_files = False
    all_results = pickle.load(stats_file)

In [ ]:
safety_pool_results = {path: res for path, res in all_results.items() if "Safety Pool" in path}
openscenario_results = {path: res for path, res in all_results.items() if "openscenario-v1.1.1" in path}
esmini_results = {path: res for path, res in all_results.items() if "esmini" in path}

results_to_analyze = all_results

conversions_to_analyze = {
    scenario_path: result.get_result() for scenario_path, result in all_results.items() if result.without_exception
}

# General Statistics

In [ ]:
analyze_statistics(results_to_analyze)

# Exceptions and tracebacks

In [ ]:
print_exception_tracebacks(results_to_analyze, compressed=True)

In [ ]:
print_exception_tracebacks_for_analyzer(results_to_analyze, EAnalyzer.DRIVABILITY, compressed=True)

In [ ]:
print_exception_tracebacks_for_analyzer(results_to_analyze, EAnalyzer.STL, compressed=True)

In [ ]:
print_exception_tracebacks_for_analyzer(results_to_analyze, EAnalyzer.SPOT, compressed=True)

# Plots

In [ ]:
plot_sim_times(results_to_analyze, path="sim_times.png", n_bins=40)
# Plot multiple sim times in one plot
# plot_sim_times([results_to_analyze, results_to_analyze], path="sim_times_all.png", label=["label 1", "label 2"])

In [ ]:
plot_num_obstacles(results_to_analyze, path="num_obstacles.png")

In [ ]:
plot_exec_times(results_to_analyze, EAnalyzer.DRIVABILITY, path="drivability_exec.png")

In [ ]:
plot_exec_times(results_to_analyze, EAnalyzer.STL, path="stl_exec.png")

In [ ]:
plot_exec_times(results_to_analyze, EAnalyzer.SPOT, path="spot_exec.png")

In [ ]:
if do_plot_scenarios:
    plot_scenarios(results_to_analyze)